In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

fpath = '../example/data/data_processed_1prod_full.json'
df = pd.read_json(fpath, lines=True)
docs = df['reviewText'].astype(str)

In [2]:
from orangecontrib.argument.miner.topic import ArguTopic, chunker

chunks = chunker(docs)
chunks.head()

/home/jqi/Workspace/orange3-argument/venv/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/jqi/Workspace/orange3-argument/venv/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/jqi/Workspace/orange3-argument/venv/lib/python3.10

,doc_id,chunk
0,0,I always get a half size up in my tennis shoes .
1,0,For some reason these feel to big in the heel ...
2,1,walked 3 hours with no problem
3,1,Put them on and !
4,1,Love them !


In [3]:
topic_model = ArguTopic()

In [4]:
topics, probs = topic_model.fit_transform_reduce_outliers(chunks['chunk'])

In [8]:
topic_model.get_topic_table().head()

,Topic,Count,Name,keyword,keyword_scores
0,-1,25,-1_excelente_hate_footbed_advice,"[excelente, hate, footbed, advice, crosstraine...","[1.1161912014785589, 0.9126471656932069, 0.912..."
1,0,105,0_sneakers_boot_cross_training,"[sneakers, boot, cross, training, sneaker, cam...","[0.29494511829404885, 0.29358364964910877, 0.2..."
2,1,108,1_fits_true_expected_fit,"[fits, true, expected, fit, perfect, glove, ne...","[0.48886861268181964, 0.47091362546608856, 0.4..."
3,2,76,2_favorite_liked_absolute_love,"[favorite, liked, absolute, love, fan, turned,...","[0.6729249092680984, 0.5758006153225247, 0.538..."
4,3,74,3_half_ordered_order_size,"[half, ordered, order, size, brand, smaller, l...","[0.5002691860685591, 0.45959737512328774, 0.42..."


In [7]:
from orangecontrib.argument.miner.topic import merger

docs_topics = merger(docs, chunks["doc_id"], topics, topic_model.get_topics(), 10)
docs_topics.head()

,doc,topic,keyword,keyword_scores
0,I always get a half size up in my tennis shoes...,"[3, 9]","[big, half, ordered, narrow, order, size, hard...","[0.7012978034718479, 0.5002691860685591, 0.459..."
1,Put them on and walked 3 hours with no problem...,"[23, 23, 2, 8]","[wore, able, times, changed, day, wearing, row...","[1.307562741066975, 1.261437709731051, 1.24419..."
2,excelente,[-1],[],[]
3,The shoes fit well in the arch area. They are ...,"[15, 9, 9, 22]","[narrow, hard, wider, wide, make, constructed,...","[0.8905823032945349, 0.8342294489739017, 0.797..."
4,Tried them on in a store before buying online ...,"[12, 0, 4, 0, 5]","[loves, daughter, purchase, satisfied, gift, w...","[0.984816198069362, 0.9329566983097395, 0.8221..."


In [8]:
import copy
import numpy as np

full_topics = copy.deepcopy(topic_model.get_topics())
full_topics.pop(-1)

size = topic_model.top_n_words
keywords = []
keyword_scores = []
for i in full_topics:
    full_topic = full_topics[i]
    keywords += [w[0] for w in full_topic]
    keyword_scores.append([w[1] for w in full_topic]) 
keyword_scores = np.array(keyword_scores)

In [9]:
# Merging topics by distribution may decrease the performance though
i = 7
result = keyword_scores * probs[i][:, np.newaxis]
df_doc_keyword = pd.DataFrame({
    "keyword": keywords, 
    "score": result.flatten()
})
df_doc_keyword = df_doc_keyword.groupby('keyword', as_index=False).max().sort_values(
    by="score", ascending=False).reset_index(drop=True)
print(docs.loc[i]['doc_id'])
print(docs.loc[i]['chunk'])
print("Topic %i" % topics[i])
for w in topic_model.get_topic(topics[i]):
    print(w)
df_doc_keyword.loc[0:9]

3
The shoes fit well in the arch area .
Topic 22
('arch', 0.8003347630850949)
('support', 0.6964831108214038)
('outside', 0.6054996839248983)
('decent', 0.549647492998692)
('kind', 0.5102821650788569)
('twinge', 0.4961268225201613)
('ripped', 0.4961268225201613)
('flimsy', 0.4961268225201613)
('higher', 0.4961268225201613)
('provided', 0.4961268225201613)


,keyword,score
0,grommets,0.016436
1,true,0.014916
2,laces,0.014784
3,photo,0.014784
4,dark,0.013420
5,left,0.012459
6,coming,0.012459
7,shown,0.012459
8,lighter,0.012113
9,squeaking,0.012113
